# Christian's Notebook
This is a place where I will create code fragments for the project.

In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import Levenshtein

In [ ]:
chart_art = """
____|\_______________|\\_______________|_______'__|__`___|_____|___|__________
____|/___3_|________@'_\|__|_____|_____|___|___|__|__|___|_|__@'___|___|___|__
___/|____-_|____________|__|_____|____@'___|__@'_@'_@'___|_|______@'___|___|__
__|_/_\__4_|___|_______@'__|____O'_________|____________O'_|__________@'___|__
___\|/_____|___|___________|_______________|_______________|_______________|__
"""


Notes:
Manually took out some columns

In [ ]:
# all_data = pd.read_excel(r"./Data/Original-Dataset.xlsx", nrows=21527)

In [ ]:
# num_of_rows = len(all_data.index)

# slice_index_1 = num_of_rows//3
# slice_index_2 = slice_index_1*2
# slice_index_3 = slice_index_1*3

# print(num_of_rows)
# print(slice_index_1, slice_index_2, slice_index_3)

# slice_1 = all_data[0: slice_index_1]
# slice_2 = all_data[slice_index_1: slice_index_2]
# slice_3 = all_data[slice_index_2: num_of_rows+1]

# slice_1.to_excel(r"./Data/dataset1.xlsx", index=False, header=True)
# slice_2.to_excel(r"./Data/dataset2.xlsx", index=False, header=True)
# slice_3.to_excel(r"./Data/dataset3-1.xlsx", index=False, header=True)
# slice_3.to_excel(r"./Data/dataset3-2.xlsx", index=False, header=True)

In [ ]:
def damerau_levenshtein_distance(str1, str2):
    return Levenshtein.distance(str1, str2)

def jaro_winkler_similarity(str1, str2):
    return Levenshtein.jaro_winkler(str1, str2)

# Example usage
string1 = "R  a p"
string2 = "rap"

distance = damerau_levenshtein_distance(string1, string2)
similarity = jaro_winkler_similarity(string1, string2)

print("Damerau-Levenshtein distance:", distance)
print("Jaro-Winkler similarity:", similarity)


In [ ]:
datasets = []
list_of_indexes = ["song_name", "key", "mode", "time_signature"]

try:
    #import all three xls files, setting it to 1 for now due to run time
    for i_index in range(1, 4):
        tmp_data = []
        for j_index in range(1,3):
            fragment_data = pd.read_excel(f"./Data/dataset{i_index}-{j_index}.xlsx")
            tmp_data.append(fragment_data)

        dataset = pd.merge(tmp_data[0], tmp_data[1], left_index=True, right_index=True)
        
        # clean DataFrame
        dataset.drop_duplicates()
        dataset["song_name"] = dataset["song_name"].str.replace(r'[^\x00-\x7F]+', '')
        dataset = dataset[dataset.song_name.str.match("[a-zA-Z0-9]+") & dataset.song_name.notna()]
        dataset = dataset[~dataset.song_name.str.match("[0-9]+\s[0-9]+")]
        dataset = dataset[~dataset.song_name.str.match("[0-9]+:[0-9]+")]
        datasets.append(dataset)
except FileNotFoundError as e:
    print(f"{type(e)}\nExiting with error.")
    exit()

df = [datasets[i_index].set_index(list_of_indexes) for i_index in range(len(datasets))]
group_25_song_database = pd.concat([df[0], df[1], df[2]])

In [ ]:
songs_sorted = sorted(group_25_song_database.index.get_level_values(0).to_list())

In [ ]:
with open("songs.txt", "w") as songs_data:
    for song in songs_sorted:
        songs_data.write(f"{song}\n")

In [ ]:
# data = ['00101111 01001101 01001001 01000001', '1 thing', '01:00:00', 'shake it off', 'f.t.w', 'Ten (å¤©)', 'å¤©', '1:1', '', ' ' , 'a3', 'shake if off', 'Canal St. (feat. Bones)', 'Ð“Ñ€ÑƒÑÑ‚Ð½Ð°Ñ', np.nan]
# names = pd.DataFrame(data, columns=["song_name"])
# names["song_name"] = names["song_name"].str.replace(r'[^\x00-\x7F]+', '')
# names = names[names.song_name.str.match("[a-zA-Z0-9]+") & names.song_name.notna()]
# print(names)
# names = names[~names.song_name.str.match("[0-9]+\s[0-9]+")]
# names = names[~names.song_name.str.match("[0-9]+:[0-9]+")]
# names

In [ ]:
list_of_songs = list(group_25_song_database.index.levels[0])
list_of_genres = list(group_25_song_database.genre.unique())

group_25_song_database['total_beats'] = (group_25_song_database['tempo'] * group_25_song_database['duration_ms']*0.001).astype(int)
group_25_song_database['boringness'] = (group_25_song_database['loudness'] + group_25_song_database['tempo'] + (group_25_song_database['energy']*100) + (group_25_song_database['danceability']*100))

In [ ]:
group_25_song_database.head()

In [ ]:
sub_genre_df = group_25_song_database.loc[group_25_song_database["genre"] == "Pop"]
df = sub_genre_df.loc[:, ["boringness", "total_beats"]]
beats = df["total_beats"].values.tolist()
boringness = df["boringness"].values.tolist()
boringness

In [48]:
data = {'First Name': ['Tom', 'Tom', 'krish', 'jack'],
        'Last Name': ['Smith', 'Smith', 'doug', 'jack'],
        'Gender': ['Male', 'Female', 'Male', 'Female'],
        'Age': [20, 20, 19, 18]}
  
# Create DataFrame
df = pd.DataFrame(data)
df2 = df.set_index(['First Name', 'Last Name'])

In [49]:
df

,First Name,Last Name,Gender,Age
0,Tom,Smith,Male,20
1,Tom,Smith,Female,20
2,krish,doug,Male,19
3,jack,jack,Female,18


In [51]:
df2.loc[df2.Age == 20][:1]

,,Gender,Age
First Name,Last Name,,
Tom,Smith,Male,20


In [37]:
idx = pd.IndexSlice
df2.loc[idx[["Tom"], :], :][0]

KeyError: 0

In [ ]:
idx = pd.IndexSlice
df3 = df2.loc[idx[:,"Smith"], :]
df3[df3.Age==21]

In [ ]:
def find_closest_target_value(numbers, target):
    closest = None
    min_diff = float('inf')

    for num in numbers:
        if num != target:
            diff = abs(num - target)
            if diff < min_diff:
                min_diff = diff
                closest = num

    return closest

In [ ]:
user_song = "Shake It Off"
user_genre = "Pop"
user_boringness = 297.906

"""
{
    'danceability': 0.648,
    'energy': 0.785,
    'loudness': -5.414,
    'speechiness': 0.165,
    'acousticness': 0.0561,
    'instrumentalness': 0.0,
    'liveness': 0.148,
    'valence': 0.943,
    'tempo': 160.02,
    'duration_ms': 219200.0,
    'total_beats': 35076.0,
    'boringness': 297.906
 }

"""

closest_boringess_value = find_closest_target_value(boringness, user_boringness)
sub_genre_df = group_25_song_database.loc[group_25_song_database["genre"] == "Pop"]
user_df = group_25_song_database.loc["Shake It Off"]
user_df = user_df[user_df.genre == user_genre]
song_attributes = [a for a in user_df.index.to_list()[0]]
song_key, song_mode, time_sig = song_attributes

idx = pd.IndexSlice
print(type(closest_boringess_value))
suggested_df = sub_genre_df[sub_genre_df.boringness == closest_boringess_value]
suggested_df
# user_df.boringness.values
# sugg_df = df[df.boringness.value == result1[0] & df.total_beats.value == result2[0]]

# df[df.boringness.values == closest_boringess_value]
# df[df.total_beats.values == closest_beat_value]

In [ ]:
group_25_song_database.describe()

In [ ]:
column_names = list(group_25_song_database.columns)
column_names.remove("genre")
database_describe = group_25_song_database.describe()
characteristics_mean = {}

for column_name in column_names:
    characteristic = group_25_song_database.describe()[column_name]
    characteristics_mean[column_name] = characteristic[characteristic.index == "mean"][0]

characteristics_mean

In [ ]:
user_song_name = "Shake It Off"
song_genre = "Pop"

user_df = group_25_song_database.loc[user_song_name]
user_df = user_df[user_df.genre == song_genre]
user_val = {}

for column_name in column_names:
    user_val[column_name] = float(user_df.loc[:, column_name].values)

user_val

In [ ]:
bar_user = {}
bar_data = {}

for key, value in user_val.items():
    bar_user[key] = value

for key, value in characteristics_mean.items():
    bar_data[key] = value


In [ ]:
unwanted_data = ["tempo"]
big_values = []
all_characteristics = list(bar_user.keys())

for key, value in characteristics_mean.items():
    if (value > 1 or value < 0) and key not in unwanted_data:
        big_values.append(key)

In [ ]:
# set width of bar
barWidth = 0.25
fig, ax = plt.subplots(figsize =(12, 8))

for characteristic in all_characteristics:
    if characteristic in unwanted_data or characteristic in big_values:
        bar_user.pop(characteristic)
        bar_data.pop(characteristic)

bar_user_values = list(bar_user.values())
x_tick_labels = list(bar_user.keys())
bar_data_values = list(bar_data.values())
br1 = np.arange(len(bar_user))
br2 = [x + barWidth for x in br1]

# Make the plot
ax.bar(br1, bar_data_values, color ='#cf0a2c', width = barWidth, edgecolor ='grey', label ='data')
ax.bar(br2, bar_user_values, color = '#1db954', width = barWidth, edgecolor ='grey', label ='user')

# Adding Xticks
ax.set_xlabel('Audio Features', fontweight ='bold', fontsize = 15)
ax.set_ylabel('Weighted Value', fontweight ='bold', fontsize = 15)
ax.set_xticks([r + barWidth for r in range(len(x_tick_labels))])
ax.set_xticklabels(x_tick_labels, rotation=65)
ax.set_title(f"{user_song_name} Attributes Against Dataset", fontweight ='bold', fontsize = 20)

plt.legend()
plt.show()

In [ ]:
bar_user = {}
bar_data = {}

for key, value in user_val.items():
    bar_user[key] = value

for key, value in characteristics_mean.items():
    bar_data[key] = value

In [ ]:
group_25_song_database.hist(figsize=(15,15),
                            grid=False,
                            bins=10,
                            color='#1db954',
                            ec='#121212',
                            legend=True)

In [ ]:
x = group_25_song_database.loc[:, :]
x = x.drop(columns="genre")
y = list_of_genres

In [ ]:
import seaborn as sns

k = 0
plt.figure(figsize=(40, 30))

for i in x.columns:
    plt.subplot(4, 4, k+1)
    sns.distplot(x[i], color='#1db954', hist_kws=dict(edgecolor="#121212", linewidth=1))
    plt.xlabel(i, fontsize=24)
    k +=1

In [ ]:
"""
    Make this automated instead of hard coding
    Find max value and choose it to "explode"
"""

colors = ["#1db594", "#ff59c7", "#a0a1cb", "#f31511", "#7e93ba", "#5d3fd3", "#d1ad00", "#e5e1e1"]

explode = [0 for _ in range(len(list_of_genres))]
user_song_genre_index = list_of_genres.index(song_genre)
explode[user_song_genre_index] = 0.1
genre_grouped = np.array(group_25_song_database.groupby(["genre"])["genre"].count())

plt.pie(genre_grouped, labels=list_of_genres, startangle=0, autopct='%1.2f%%', shadow=True, colors=colors, explode=explode)
plt.title("Database Genre Pie Chart", fontsize=20, fontweight="bold", color="#121212")

plt.show()